In [ ]:
# Feature selection: try SHAP/permutation importance on a simpler model first
# (e.g., LightGBM) to reduce noise, then feed selected features to LSTM.

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import tensorflow as tf
from keras import layers, callbacks, models, Sequential, Input

# ---------------------
# Configurable parameters
# ---------------------
#CSV_PATH   = "new_data.csv"      # path to your uploaded file
TARGET_COL = "GC_F_Close"        # predict tomorrow's close for gold futures
#DATE_COL   = "Date"
LOOKBACK   = 30               # number of past days in each sequence
BATCH_SIZE = 16
EPOCHS     = 100
VAL_PATIENCE = 8                 # early stopping patience
LR_REDUCE_PATIENCE = 4
LAGS = [1, 5, 10, 20]

"""# ---------------------
# 1) Load & basic cleaning
# ---------------------
df = pd.read_csv(CSV_PATH)

# Ensure datetime and sort chronologically # pourquoi if, try ?
if DATE_COL in df.columns:
    df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
    df = df.sort_values(DATE_COL).reset_index(drop=True)

# Drop rows where target is missing
df = df.dropna(subset=[TARGET_COL])

# Create tomorrow's target by shifting -1 (next day)
df["target_tomorrow"] = df[TARGET_COL].shift(-1)

# After shift, last row has NaN target; drop it
df = df.dropna(subset=["target_tomorrow"]).reset_index(drop=True)

# Lag features: add past values of the target as exogenous inputs # pas compris cette partie
#for lag in LAGS:
    #df[f"{TARGET_COL}_lag{lag}"] = df[TARGET_COL].shift(lag)"""

# ---------------------
# 2) Feature selection & imputation
# ---------------------
# Inputs = all columns except DATE_COL and TARGET_COL; exclude helper columns too
exclude = {"GC_F_Close"}
feature_cols = [c for c in df.columns if c not in exclude]

# Time-series friendly imputation: forward-fill then back-fill
X_raw = df[feature_cols]

#.copy().ffill().bfill()

# Target vector
y_raw = df["GC_F_Close"].values.astype(np.float32)

# ---------------------
# 3) Chronological split: train / val / test
# ---------------------
n = len(df)
train_end = int(n * 0.70)
val_end   = int(n * 0.85)

X_train_raw = X_raw.iloc[:train_end].copy()
X_val_raw   = X_raw.iloc[train_end:val_end].copy()
X_test_raw  = X_raw.iloc[val_end:].copy()

y_train = y_raw[:train_end]
y_val   = y_raw[train_end:val_end]
y_test  = y_raw[val_end:]

# ---------------------
# 4) Scale features (fit on train only)
# ---------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_val_scaled   = scaler.transform(X_val_raw)
X_test_scaled  = scaler.transform(X_test_raw)

n_features = X_train_scaled.shape[1]

# ---------------------
# 5) Build rolling windows (LSTM sequences)
# ---------------------

def make_sequences(X, y, lookback):
    #Build sequences of shape (num_samples, lookback, num_features) and aligned targets.
    #Each sequence uses X[t-lookback:t] to predict y[t].

    X_seq, y_seq = [], []
    for t in range(lookback, len(X)):
        X_seq.append(X[t - lookback:t])
        y_seq.append(y[t])
    return np.asarray(X_seq, dtype=np.float32), np.asarray(y_seq, dtype=np.float32)

X_tr_seq, y_tr_seq = make_sequences(X_train_scaled, y_train, LOOKBACK)
X_va_seq, y_va_seq = make_sequences(X_val_scaled,   y_val,   LOOKBACK)
X_te_seq, y_te_seq = make_sequences(X_test_scaled,  y_test,  LOOKBACK)

print(f"Train seq shape: {X_tr_seq.shape}, Val: {X_va_seq.shape}, Test: {X_te_seq.shape}")

# ---------------------
# 6) Define the LSTM model
# ---------------------
def build_model(n_features, lookback):
    model = Sequential()
    model.add(Input(shape=(lookback, n_features)))

    # LSTM principal
    model.add(layers.LSTM(512, return_sequences=True))
    model.add(layers.LSTM(units=256,return_sequences=False))

    # Dense non linéaire
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(32, activation="relu"))
    # Sortie
    model.add(layers.Dense(1))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(3e-4),
        loss='mse',
        metrics=['mae']
    )

    return model


model = build_model(n_features,LOOKBACK)
model.summary()

# ---------------------
# 7) Callbacks
# ---------------------
ckpt_path = "best_lstm_gold.keras"
cbs = [
    callbacks.ModelCheckpoint(ckpt_path, monitor="val_loss", save_best_only=True, verbose=1),
    callbacks.EarlyStopping(monitor="val_loss", patience=VAL_PATIENCE, restore_best_weights=True, verbose=1),
    callbacks.ReduceLROnPlateau(monitor="val_loss", patience=LR_REDUCE_PATIENCE, factor=0.5, min_lr=1e-5, verbose=1),
]

# ---------------------
# 8) Train
# ---------------------
history = model.fit(
    X_tr_seq, y_tr_seq,
    validation_data=(X_va_seq, y_va_seq),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=False,        # important for time series
    callbacks=cbs,
    verbose=1
)

# ---------------------
# 9) Evaluate on test
# ---------------------
y_pred_test = model.predict(X_te_seq).squeeze()

mae  = mean_absolute_error(y_te_seq, y_pred_test)
rmse = sqrt(mean_squared_error(y_te_seq, y_pred_test))

def mape(a, f):
    return np.mean(np.abs((a - f) / np.clip(np.abs(a), 1e-8, None))) * 100

mape_val = mape(y_te_seq, y_pred_test)

print(f"Test MAE:  {mae:,.4f}")
print(f"Test RMSE: {rmse:,.4f}")
print(f"Test MAPE: {mape_val:,.2f}%")

# ---------------------
# 10) Plot predictions vs actual
# ---------------------
plt.figure(figsize=(12,4))
plt.plot(y_te_seq, label="Actual (tomorrow GC=F_Close)")
plt.plot(y_pred_test, label="Predicted", alpha=0.8)
plt.title("LSTM prediction: tomorrow GC=F_Close")
plt.xlabel("Test time steps")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.savefig("lstm_gold_test_plot.png", dpi=140)

# ---------------------
# 11) Save predictions and scaler
# ---------------------
pd.DataFrame({
    "y_true": y_te_seq,
    "y_pred": y_pred_test
}).to_csv("lstm_gold_test_predictions.csv", index=False)

# Save scaler for inference consistency
import joblib
joblib.dump(scaler, "feature_scaler.pkl")

print("Artifacts saved: best_lstm_gold.keras, lstm_gold_test_plot.png, lstm_gold_test_predictions.csv, feature_scaler.pkl")
